In [ ]:
#Learning curves visualization
import json
import matplotlib.pyplot as plt
import os
import glob
from matplotlib import rc

def load_results(base_dirs, method_name, model_names=None):
    """
    Load results from multiple directories
    
    Args:
        base_dirs (list): List of directory paths to search in
        method_name (str): Method name to filter results ('2s' or 'spo+')
        model_names (list, optional): List of model names to filter. If None, load all models.
    
    Returns:
        dict: Dictionary with results organized by model name
    """
    results = {}
    
    for base_dir in base_dirs:
        # Update pattern to match both '2s' and 'two_stage'
        if method_name == 'two_stage':
            json_pattern = os.path.join(base_dir, 'logs', f'*_2s_results.json')
        else:
            json_pattern = os.path.join(base_dir, 'logs', f'*_{method_name}_results.json')
        json_files = glob.glob(json_pattern)
        
        for json_file in json_files:
            with open(json_file, 'r') as f:
                data = json.load(f)
                
            model_name = data['model_name']
            
            # Map model names from file to display names
            if model_names and model_name not in model_names:
                continue
                
            if model_name not in results:
                results[model_name] = []
            
            results[model_name].append({
                'loss_log': data['loss_log'],
                'dir': base_dir
            })
    
    return results

def plot_combined_learning_curves(base_dirs, model_names=None, save_path=None):
    """
    Plot learning curves for both methods in one figure with simplified legend
    
    Args:
        base_dirs (list): List of directory paths
        model_names (list, optional): List of model names to plot
        save_path (str, optional): Path to save the figure
    """
    # Set up LaTeX-style plotting
    rc('font', **{'family': 'serif'})
    rc('text', usetex=True)
    rc('figure', figsize=(15, 8))
    rc('font', size=20)
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Define colors for different directories
    colors = {
        'max_vis': '#1f77b4',    # Blue
        'max_prec': '#2ca02c',   # Green
        'min_time': '#d62728',   # Red
        'max_wind': '#9467bd',   # Purple
        'max_danger': '#ff7f0e'  # Orange
    }
    
    # Define line styles and display names for different models
    styles = {
        'mlp': '-',
        'simple_regression': '--'
    }
    
    model_display_names = {
        'mlp': 'MLP',
        'simple_regression': 'Linear Regression'
    }
    
    # Plot both SPO+ and Two-Stage results
    methods = {'spo+': (ax1, 'SPO+'), 'two_stage': (ax2, 'Two-Stage')}
    legend_handles = []
    legend_labels = []
    
    for method_name, (ax, display_name) in methods.items():
        results = load_results(base_dirs, method_name, model_names)
        
        for model_name, model_results in results.items():
            for result in model_results:
                dir_name = os.path.basename(result['dir'])
                line = ax.plot(result['loss_log'], 
                        color=colors[dir_name],
                        linestyle=styles[model_name],
                        alpha=0.8)[0]
                
                # Only add to legend for the first plot (SPO+)
                if method_name == 'spo+':
                    legend_handles.append(line)
                    legend_labels.append(f"{dir_name} ({model_display_names[model_name]})")
    
    # Set titles and labels
    ax1.set_title('SPO+ Learning Curves')
    ax2.set_title('Two-Stage Learning Curves')
    
    for ax in [ax1, ax2]:
        ax.set_xlabel('Iterations')
        ax.set_ylabel('Loss')
        ax.grid(True, alpha=0.3)
    
    # Add single legend between the plots
    fig.legend(legend_handles, legend_labels,
              loc='center left',
              bbox_to_anchor=(1.0, 0.5))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()

# Example usage
if __name__ == "__main__":
    directories = [
        "max_vis",
        "max_prec",
        "min_time",
        "max_wind",
        "max_danger"
    ]
    
    plot_combined_learning_curves(
        base_dirs=directories,
        model_names=["mlp", "simple_regression"],
        save_path="learning_curves_combined.pdf"
    )

In [ ]:
#Normalized regret visualization
import numpy as np

def load_regret_results(base_dirs):
    """
    Load regret results from multiple directories
    
    Args:
        base_dirs (list): List of directory paths to search in
    
    Returns:
        dict: Dictionary with regret results organized by scenario, method, and model
    """
    # Define valid combinations (excluding two_stage + mlp)
    valid_combinations = [
        ('spo+', 'mlp'),
        ('spo+', 'simple_regression'),
        ('two_stage', 'simple_regression')
    ]
    
    results = {combo: {} for combo in valid_combinations}
    
    for base_dir in base_dirs:
        scenario_name = os.path.basename(base_dir)
        
        for method, model in valid_combinations:
            # Handle both '2s' and 'two_stage' in filenames
            pattern = '2s' if method == 'two_stage' else method
            json_pattern = os.path.join(base_dir, 'logs', f'*_{pattern}_results.json')
            json_files = glob.glob(json_pattern)
            
            for json_file in json_files:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                    
                if data['model_name'] != model:
                    continue
                
                key = (method, model)
                if scenario_name not in results[key]:
                    results[key][scenario_name] = []
                
                # Store the entire regret log for this run
                results[key][scenario_name].extend(data['loss_log_regret'])
    
    return results

def plot_regret_boxplots(base_dirs, save_path=None):
    """
    Create boxplot visualization of regrets across different scenarios
    
    Args:
        base_dirs (list): List of directory paths
        save_path (str, optional): Path to save the figure
    """
    # Set up LaTeX-style plotting
    rc('font', **{'family': 'serif'})
    rc('text', usetex=True)
    rc('figure', figsize=(15, 8))
    rc('font', size=20)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Load results
    results = load_regret_results(base_dirs)
    
    # Define colors and offsets for different combinations
    config = {
        ('spo+', 'mlp'): {
            'color': '#1f77b4',  # Blue
            'offset': -0.25,
            'label': 'MLP + SPO+'
        },
        ('spo+', 'simple_regression'): {
            'color': '#2ca02c',  # Green
            'offset': 0,
            'label': 'Linear Regression + SPO+'
        },
        ('two_stage', 'simple_regression'): {
            'color': '#d62728',  # Red
            'offset': 0.25,
            'label': 'Linear Regression + Two-Stage'
        }
    }
    
    # Get unique scenarios and sort them
    scenarios = sorted(list(results[('spo+', 'mlp')].keys()))
    
    # Create boxplots for each combination
    boxplots = {}
    for combo, cfg in config.items():
        # Prepare data for this combination
        combo_data = [results[combo][scenario] for scenario in scenarios]
        
        # Create boxplot
        positions = np.array(range(len(scenarios))) + cfg['offset']
        bp = ax.boxplot(combo_data,
                       positions=positions,
                       widths=0.2,  # Increased width since we have fewer boxes
                       showfliers=False,
                       patch_artist=True)
        
        # Style the boxplot
        define_box_properties(bp, cfg['color'], cfg['label'])
        
        boxplots[combo] = bp
    
    # Customize the plot
    ax.set_xlabel('Scenarios')
    ax.set_ylabel('Normalized Regret')
    # ax.set_title('Regret Distribution Across Scenarios')
    
    # Set x-axis labels
    ax.set_xticks(range(len(scenarios)))
    ax.set_xticklabels([s.replace('_', ' ').title() for s in scenarios], rotation=45)
    
    # Add grid
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.set_axisbelow(True)
    
    # Adjust layout
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()

def define_box_properties(bp, color, label):
    """
    Set the color properties of a boxplot
    
    Args:
        bp: Boxplot object
        color (str): Color code
        label (str): Label for legend
    """
    plt.setp(bp['boxes'], color=color, facecolor=color, alpha=0.7)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color='white')
    
    # Add to legend
    plt.plot([], c=color, label=label)
    plt.legend( loc='upper right')

# Example usage
if __name__ == "__main__":
    directories = [
        "max_vis",
        "max_prec",
        "max_wind",
        "max_danger",
        "min_time"
    ]
    
    plot_regret_boxplots(
        base_dirs=directories,
        save_path="regret_boxplots.pdf"
    )

In [ ]:
#Flight trajectory visualization
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import shapely.geometry as sgeom

def get_full_trajectory(date, callsign, parquet_dir='EGKK'):
    """Get full trajectory for a specific flight from parquet files"""
    date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
    file_pattern = f"flight_data_{date_str}*.parquet"
    matching_files = glob(os.path.join(parquet_dir, file_pattern))
    
    if not matching_files:
        return None
    
    df = pd.read_parquet(matching_files[0])
    flight_data = df[df['callsign'] == callsign].sort_values('time')
    
    return flight_data

def plot_trajectories(df, n_flights=10, tolerance=0.01):
    rc('font',**{'family':'serif'})
    rc('text', usetex=True)
    rc('figure', figsize=(11.69,8.27))
    # rc('figure', figsize=(15,10))
    rc('font',size = 22)
    """Plot simplified trajectories using Shapely's simplify function"""
    # plt.figure(figsize=(20, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    
    # Sample flights
    sampled_flights = df.sample(n=n_flights)
    colors = plt.cm.rainbow(np.linspace(0, 1, n_flights))
    
    for (_, flight), color in zip(sampled_flights.iterrows(), colors):
        full_traj = get_full_trajectory(flight['time'], flight['callsign'])
        
        if full_traj is not None and not full_traj.empty:
            # Create LineString and simplify
            simplified_track = sgeom.LineString(zip(full_traj['lon'], full_traj['lat'])).simplify(tolerance)
            
            # Add trajectory line
            ax.add_geometries([simplified_track], ccrs.PlateCarree(),
                            facecolor='none', edgecolor=color, linewidth=1, alpha=0.1)
            
            # # Plot start and end points
            # ax.plot(full_traj['lon'].iloc[0], full_traj['lat'].iloc[0], 
            #        'o', color=color, markersize=5, transform=ccrs.PlateCarree())
            # ax.plot(full_traj['lon'].iloc[-1], full_traj['lat'].iloc[-1], 
            #        's', color=color, markersize=5, transform=ccrs.PlateCarree())
    
    # Add Gatwick Airport
    airport_lat, airport_lon = 51.1537, -0.1821
    ax.plot(airport_lon, airport_lat, '*', color='red', markersize=15, 
            transform=ccrs.PlateCarree(), label='EGKK')
    
    # Set map extent and add gridlines
    ax.set_extent([df['lon'].min()-.5, df['lon'].max()+.5, 
                  df['lat'].min()-.5, df['lat'].max()+.5])
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                     linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    plt.legend()
    # plt.title('Sample Flight Trajectories')
    plt.tight_layout()
    plt.show()


df = pd.read_csv('filtered_flight_data_all_50.csv')
plot_trajectories(df, n_flights=10, tolerance=0.01)

In [ ]:
#Area of interests
import pandas as pd


df = pd.read_csv('filtered_flight_data_all_50.csv')

# Create a new figure with a larger size for better visibility
plt.figure(figsize=(20, 10))

# Set up the map projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the points
scatter = ax.scatter(df['lon'], df['lat'], c=df['transit_time'], 
                     cmap='viridis', transform=ccrs.PlateCarree(),
                     s=10, alpha=0.6)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Transit Time (seconds)', rotation=270, labelpad=15)
# Read waypoints from CSV file
waypoints_df = pd.read_csv('data/waypoints.csv')

# Plot waypoints
for _, waypoint in waypoints_df.iterrows():
    ax.plot(waypoint['Longitude'], waypoint['Latitude'], 
            marker='^', markersize=8, color='red', 
            transform=ccrs.PlateCarree())
    ax.text(waypoint['Longitude']+0.05, waypoint['Latitude']-0.01, 
            waypoint['Name'], fontsize=8, 
            ha='right', va='bottom', transform=ccrs.PlateCarree())

# Add legend for waypoints
ax.plot([], [], marker='^', markersize=8, color='red', 
        linestyle='None', label='Waypoints')
ax.plot([], [], marker='*', markersize=8, color='blue', 
        linestyle='None', label='EGKK')
airport_lat, airport_lon = 51.1537, -0.1821

# Visualize the airport
ax.plot(airport_lon, airport_lat, marker='*', markersize=10, color='blue', 
        transform=ccrs.PlateCarree())
ax.text(airport_lon, airport_lat, 'EGKK', fontsize=10, 
        ha='right', va='bottom', transform=ccrs.PlateCarree())
plt.legend()

# Set the map extent to focus on the data
ax.set_global()
ax.set_extent([df['lon'].min()-.5, df['lon'].max()+.5, 
               df['lat'].min()-.5, df['lat'].max()+.5])

# Add gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# plt.title('Flight Data: Transit Time Visualization')
plt.tight_layout()
plt.show()